<a href="https://colab.research.google.com/github/yoonyoung27/prac_class/blob/main/personal_project_M1F4_%EC%A7%80%EC%9C%A4%EC%98%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.dacon.io/competitions/official/235713/overview/description/